In [1]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D, Dropout

model=Sequential()

model.add(Conv2D(256, (3, 3),input_shape=(50, 50, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dense(8, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 256)       2560      
_________________________________________________________________
activation (Activation)      (None, 48, 48, 256)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 128)       295040    
_________________________________________________________________
activation_1 (Activation)    (None, 22, 22, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0

In [2]:
model.load_weights('../res/Models/modelAgeGender-0.7314285635948181.h5')

In [3]:
import cv2, pickle
import dlib, os
import numpy as np
from numpy import random

In [4]:
ages = []
prio_ages = []
i=0

In [5]:
def most_frequent(List):
    try:
        counter = 0
        num = List[0] 

        for i in List: 
            curr_frequency = List.count(i) 
            if(curr_frequency> counter): 
                counter = curr_frequency 
                num = i 
        return num
    except:
        pass

In [6]:
def play_video(prio_min):
    ad_dirs = os.listdir("../res/Data/Ads")
    for _dir in ad_dirs:
        if(str(_dir)==str(category_dict[prio_min])):
            ads = os.listdir(f"../res/Data/Ads/{_dir}")
            num = random.randint(len(ads))
            print(f"Playing - ../res/Data/Ads/{_dir}/{ads[num]}")
    print(f"High Priority Goes to : {category_dict[prio_min]}")

In [7]:
cam = cv2.VideoCapture('../res/test_data/videos/vid.mp4')#rtsp://192.168.1.6/live/ch00_1 'https://192.168.1.5:8080/video' '../res/test_data/videos/vid.mp4'

dict_file = open("../res/Data/cat_dict.pkl", "rb")
category_dict = pickle.load(dict_file)

face_detector = dlib.get_frontal_face_detector()
margin = 20

In [8]:
while(True):
    ret, feed = cam.read()
    feed = cv2.resize(feed, (852, 480))
    
    faces = face_detector(feed)
    
    try:
        for face in faces:

            x1 = (face.left()) - margin
            y1 = (face.top()) - margin
            x2 = (face.right()) + margin
            y2 = (face.bottom()) + margin

            test_img = feed[y1:y2, x1:x2]
            s_test = test_img

            test_img = cv2.resize(test_img, (50,50))
            test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
            test_img = test_img/255
            test_img = test_img.reshape(1, 50, 50, 1)

            results = model.predict(test_img)
            label = np.argmax(results, axis=1)[0]
            acc = int(np.max(results, axis=1)[0]*100)

            #print(results,label,category_dict[label],acc)
            
            ages.append(int(label))
            
            cv2.imwrite(f"../res/Live_Face/{category_dict[label]}({acc}%).jpg", s_test)

            cv2.rectangle(feed, (x1, y1), (x2, y2), (0, 255, 0), 2)

            cv2.putText(feed, f"{category_dict[label]}({str(acc)}%)", (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

    except:
        pass
    try:
        prio = most_frequent(ages)
        
        if(i<120):
            prio_ages.append(int(prio))
            i+=1
        else:
            prio_min = most_frequent(prio_ages)
            play_video(prio_min)
            i=0
            prio_ages=[]
    except Exception as e:
        pass
        #print(e)
    ages = []
        
    cv2.imshow('LIVE_FEED',feed)
    k=cv2.waitKey(1)
    if(k==27):
        break
cv2.destroyAllWindows()

Playing - ../res/Data/Ads/13-26_Male/ad_34613434_0771e18633168399_web.mp4
High Priority Goes to : 13-26_Male
Playing - ../res/Data/Ads/13-26_Male/mediarepo_34601977_2c2621d5b3c3614d_web.mp4
High Priority Goes to : 13-26_Male
Playing - ../res/Data/Ads/13-26_Male/mediarepo_34601977_2c2621d5b3c3614d_web.mp4
High Priority Goes to : 13-26_Male
Playing - ../res/Data/Ads/13-26_Male/ad_34558545_8ef2d33c85887943_web.mp4
High Priority Goes to : 13-26_Male
Playing - ../res/Data/Ads/13-26_Male/ad_34613434_0771e18633168399_web.mp4
High Priority Goes to : 13-26_Male
